# Л4: Создание ИИ-ассистента для написания кода

В данной работе необходимо создать ИИ-ассистента, который генерирует кода в виде готового файла. У ИИ-ассистента должен быть графический интерфейс (используем Gradio)

Для доступа к LLM можно использовать:
- **Inference API** для удаленного обращения к LLM (*можно запускать большие модели, но у вас может закончиться лимит*)
- Интерфейс `pipeline` из библиотеки `transformers` для локального запуска LLM (*лимит бесконечный, но нельзя запускать большие модели*)

## 1/ Подготовка среды выполнения

***Примечание.*** Эти шаги необходимо выполнить, если вы выпорлняете работу локально на компьютере. Если используется онлайн платформу, такую как Google Colab, то пропускаете этот этап.

На данной этапе вам наобходимо подготовить виртуальное окружение и установить все необходимые библиотеки.

1. Создать и активировать (или только активировать, если ранне создавали) виртуальной окружение `python`.

Создайте на диске свою раюочую папку. Откройте терминал и перейдите в свою рабочую директорию
```
cd path/to/your/workspace
```
Далее создаем виртуальное окружение с помощью `python-venv`
```
python -m venv env
```
*активируем виртуальное окружение*
для CMD:
```
env\Scripts\activate
```
для PowerShell
```
env\Scripts\Activate.ps1
```
для bash
```
source env/bin/activate
```
**Примечание.** `env` - это название вашего виртуального окружения, назвать его можете как угодно.

После этого можем выбрать наш локальный интерпрететор pyhton, нажав на кнопку выше "Select kernel".

2. Устанавливаем все необходимые библиотеки

**Примечание.** Библиотеки установятся внутрь вашего виртуального окружения.

Нам понадобятся библиотеки Diffusers, Transformers, Accelerate.

```
pip install transformers
```
Также для работы вышеперечисленных бибилиотек потребуется PyTorch:
```
pip install torch
```
Библиотека Gradio для создания web-приложения.
```
pip install gradio
```

## 2/ Начало рабты. Создание Inference Client

Для бессерверного обращение к модели (то есть без запуска ее на своем каком-то сервере или локально) необходимо отправить запрос, используя Inference API. В результате этого запроса мы получим ответ - Inference (то есть вывод модели). Запрос можно формировать разынми способами, но в этой работе предлагается использовать Python библиотеку `huggingface_hub`. Выполните установку
```
pip install --upgrade huggingface_hub
```

Эта библиотека предоставляет модуль `InferenceClient`, с помощью которого создадим клиент, который будет формировать запрос на сервера Hugging Face. Импортируем этот модуль. Также импортируете Gradio.

In [ ]:
from huggingface_hub import InferenceClient
import gradio as gr

Есть одно НО! Для использования Serverless Inference API необходимо иметь **токен доступа**. Чтобы его получить необходимо зарегистрироваться на сайте Hugging Face, затем перейдите на [странице создания токенов](https://huggingface.co/settings/tokens/new?globalPermissions=inference.serverless.write&tokenType=fineGrained). Создайте `fine-grained` токен с областью действия `Make calls to the serverless Inference API`.

***Примечание***. Не распространяйте свой токен в публичных местах, иначе он будет скопрометирован и Hugging Face его удалит! Токен можно скопировать только один раз в моменте его создания. Сохраните его где-нибудь лично у себя и используете только в коде своего приложения когда необходимо.

Вставьте сюда сгенерированный токен.

In [ ]:
hf_token = "hf_*******************"

Для создания умного ассистента, который умеет писать код лучше нас, будем использовать LLM для генерации текста

In [ ]:
ai_model = "openai/gpt-oss-20b" # или другая LLM

Создадим клиента через класс `InferenceClient`, указав ему используемую модель и свой токен доступа:

In [ ]:
client = InferenceClient(model=ai_model, token=hf_token)

Если используете локальный запуск моделей через интерфейс `pipeline`

In [ ]:
from transformers import pipeline
ai_model = "google/gemma-3-1b-it" # берем модель попроще и полегче
model = pipeline("text-generation", model=ai_model)

## 3/ Тестирование работы модели в режиме генерации текста (`Text Generation`)
Здесь все просто: пишем запрос к ИИ (промт), в котором грамотно, четко и подробно излагаем наши требования. Например, попросим создать приложение, выводящее на графике сумму трех синусоид в реальном времени:

In [ ]:
prompt = "write one example python code to output a graph of the sum of three sinusoids"

Вызовем у нашего клиента `client` метод `text_generation()` с нашим запросом:

In [ ]:
ans = client.text_generation(prompt=prompt, max_new_tokens=1000)

Если интерфейс `pipeline`:

In [ ]:
ans = model(prompt)

Сохарним вывод модели в файл для дальнейшего прочтения

In [ ]:
with open('ans.txt', 'w+', encoding='utf-8') as file:
    file.write(ans) # для pipeline необходимо текст извлечь как ans[0]["text-generation"]

Откройте файл в текстовом редакторе (можно тут). Вы увидеть текст в разметке **Markdown**. Согласно этой разметке, код помещается между специальными символами "```". Скопируйте этот код в блок кода ниже и запустите. Должен сработать...

In [ ]:
# TODO: вставьте сюда код, который придумала модель ИИ
# возможно потребуется установить дополнительные библиотеки для работы кода, посмотрите внимательно на импорты

Скорее всего код у вас заработал! 👩‍💻 

## 4/ ЗАДАНИЕ ✅

Требуется создать приложение на основе Gradio, которое с помощью LLM генерирует программный код и выдает его в виде файла с соответствующи расширением.

Требования к приложению:

- **ВХОДНЫЕ ДАННЫЕ:** текст запроса
- **ВЫХОДНЫЕ ДАННЫЕ:** программный код, файл программного кода.
  - **ЯП:** по умолчанию просим герерировать Python-код, но вы неограничены в своих предпочтениях
- **ГРАФИЧЕСКИЙ ИНТЕРФЕЙС:**
  - текстовое поле ввода запроса
  - текстовое поле вывода кода (с поддержкой подсветки синтаксиса)
  - кнопка выгрузки файла

### 🔎 *Указанию к выполнению задания*
Ответ ИИ помимо кода содержит дополнительный поясняющий текст и т.д., **от которого нужно избавится!**. Это можено решить двумя способами:
1. Самостоятельно из всего ответа извлечь только код, котрый находится между специальными символами. Это можно сделать с помощью стандартных средств python работы со строками. Пользуйтесь google, документацией, другим ИИ, чем угодно для самостоятельного решения этой задачи
2. [ПРЕДПОЧТИТЕЛЬНО] Так составить промпт к модели, чтобы она выдала вам чистый код. Это называется **промпт-инженерия**, когда вы направляете работу LLM с помощью правильных подсказов (промптов). Например, ваш промпт может начинаться с указанием роли LLM:

```python
prompt = "Ты - senior-разработчик python с 20-ти летним стажем..."
```
Так LLM пойем кто она и активирует нужные области памяти в неоронной сети. Дале вы можеет писать саму задачу:

```python
prompt = "Ты - senior-разработчик python с 20-ти летним стажем. Напиши код на Python, который ... <описываете задачу>"
```

В конце следует дать инструкции и описать требования к ответу. Например, если мы хоим получить только чистый код без пояснений, то:
```python
prompt = "... В ответе представь исключительно код, не делай пояснения кроме комментариев в самом коде. Код заключи в формат markdown."
```

Полный промпт необязательно прописывать пользователю в интерфейсе. Постоянные части (начало, где указываем роль и конец с требованиями) - это все системные подскази, они не меняются. Поэтому их можно оставить в backend, и формировать полной промпт используюя промпт от пользователя (**где он расписывает задачу** `"Напиши код, который и т.д."`) включая его в системный промпт

## Форма отчетности
В качестве отчета по лабораторной работе вам необходимо предоставить:
1. Файл с кодом вашего приложения (Python Script или Jupyter Nonebook)
2. Скриншот работы графического web-интерфейса вашего приложения

Отчет разместить на **moodle**